In [30]:
df1 = pd.read_csv('preprocessing_실리콘벨리기사.csv')

df1.rename(columns={'0':'raw','1':'tuned'},inplace=True)

doc = df1['raw'].tolist()
sentences = doc[0].split(".")

In [31]:
import networkx as nx
import numpy as np
import pandas as pd
from konlpy.tag import Okt
okt = Okt()

from sklearn.feature_extraction.text import TfidfTransformer

In [47]:
def textrank(document):
    sentences = document[0].split('.')
    tuned_sentences = []

    for idx in range(len(sentences)) : 
        tuned_sentences.append(okt.nouns(sentences[idx]))

    tuned_sentences = [','.join(tuned_sentences[idx]).replace(',',' ') for idx in range(len(tuned_sentences))]
    vectorizer = TfidfVectorizer()
    tfidf_vector = vectorizer.fit_transform(tuned_sentences).toarray()
    similarity_graph = tfidf_vector * tfidf_vector
 
    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)

    scores = nx.pagerank(nx_graph)
    return sorted(((scores[i],s) for i,s in enumerate(sentences)),
                  reverse=True)

In [55]:
def textrank(document):
    sentences = document.split('.') # 문서를 문장으로 split
    tuned_sentences = []
    
    for idx in range(len(sentences)) : 
        tuned_sentences.append(okt.nouns(sentences[idx]))

    tuned_sentences = [','.join(tuned_sentences[idx]).replace(',',' ') for idx in range(len(tuned_sentences))]
    
    # 위에까지 문장에서 명사만 추출한 후, 명사의 나열로 각각의 문장을 대체한다.
    
    convert_to_binary_ls = CountVectorizer().fit_transform(tuned_sentences)
    tfidf_vetorized_ls = TfidfTransformer().fit_transform(convert_to_binary_ls)
    
    # 위의 두 줄의 코드 ( countervectorizer -> tf-idf transformer) 는 tf-idf vectorizer 과 같은 효과를 가진다.

    similarity_graph = tfidf_vetorized_ls * tfidf_vetorized_ls.T # 벡터의 내적을 통해 코사인 유사도 계산
 
    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph) # 그래프 위에 벡터를 올린다.

    scores = nx.pagerank(nx_graph) # 페이지 랭크 알고리즘으로 노드의 가중치를 업데이트 시켜준다.
    
    return sorted(((scores[i],s) for i,s in enumerate(sentences)),
                  reverse=True)

In [57]:
textrank(doc[0])

[(0.040816762831927554, ' 창업자들의 세부 전공을 보면 37%만 공학·컴퓨터 기술이며, 수학 전공자는 2%뿐이었다'),
 (0.03487320736779421,
  ' 요즘 실리콘밸리에서 확인되는 것은 4차 산업혁명 시대에는 예술과 인문학이 의학·공학만큼 중요하다는 사실이다'),
 (0.03339024696666714, ' 기술 관련 제품의 성공에 가장 중요한 것은 디자인"이라고 답했다'),
 (0.03282283236060363,
  ' 자녀의 직업적 성공을 위해 대학 전공으로 의학과 이공계를 우선적으로 고려하는 일은 한국이 산업화 중이던 상황에선 올바른 선택이었다'),
 (0.03272401428499746,
  '기술을 통해 놀라운 미래를 창조하고 새로운 산업혁명 시대에 리더가 되기 위해 한국에서는 공학자들과 손잡고 일할 음악가와 화가가 더 많이 필요하다'),
 (0.032391493195152554,
  ' 여기에는 컴퓨터와 인공지능, 디지털 의술, 로봇공학, 합성생물학 등 광범위한 기술이 활용된다'),
 (0.032023279817589984,
  '교육을 삶의 최우선 순위로 두고 있는 한국의 부모들은 대학 전공 가운데 의학과 공학·과학을 중시한다'),
 (0.03060327736966526, ' 이는 인문학 전공자들이 가장 잘 훈련받은 분야이다'),
 (0.02977677825410388,
  ' 교양과 인문학이 결합한 기술이야말로 가슴 벅찬 결과를 낳을 것"이라며 예술과 디자인의 중요성을 강조했다'),
 (0.029744919107691305,
  " 초소형 물질의 발전은 모든 사람이 충분히 이용할 수 있는 '태양열 저장 기술'의 새 시대를 열 것이다"),
 (0.02924623343757161,
  ' 최근 내가 그에게 "어떻게 5년여 만에 회사 주가를 450% 올릴 수 있었느냐"고 물었더니 그는 "우리 회사가 만드는 모든 제품의 디자인을 쉼 없이 고민했기 때문에 가능했다'),
 (0.028901992422112152, '실리

In [62]:
from gensim.summarization import summarize
summarize(doc[0])

'요즘 실리콘밸리에서 확인되는 것은 4차 산업혁명 시대에는 예술과 인문학이 의학·공학만큼 중요하다는 사실이다.스티브 잡스는 자신이 대학 시절 수강했던 서체(書體) 수업이 매킨토시(애플이 1984년 발표한 개인용 컴퓨터) 개발 성공에 큰 영향을 미쳤다고 말했다.\n이런 관점을 바탕으로 잡스는 세계 최고 가치를 인정받는 기업을 만들었고, 기술 산업의 새로운 표준까지 정했다.실리콘밸리에서 최근 뜨고 있는 스타 기업인 중에는 인문학 전공자들이 제법 많다.\n구인·구직 소셜 네트워킹 서비스 기업인 링크드인(LinkedIn) 창업자 리드 호프만은 철학 석사 학위 소지자이며, 수잔 보이치키 유튜브 CEO는 역사와 문학을 전공했다.\n메신저 개발 업체 슬랙(Slack)의 창업자 스튜어트 버터필드는 철학, 세계 최대 숙박 공유 기업인 에어비앤비의 설립자 브라이언 체스키는 미술을 전공했다.\n이들의 전공은 경영·회계·보건·예술·인문학 등 매우 다양했다.컴퓨터 주변기기 제조 업체 로지텍의 브랙큰 대럴 CEO도 영문학을 전공했다.'